In [ ]:
# Pour rouler dans Google Colab, executez cette cellule en premier

!git clone https://github.com/Algolab-Sherhack-2024/defi-principal.git
import sys
sys.path.insert(0,'/content/defi-principal')
!pip install -r defi-principal/requirements.txt

# Problème 3 : Partition des flottes d'autobus

Vous avez réussi à résoudre le problème de partition des flottes d’autobus en partie 2. Maintenant, nous allons complexifier le problème en ajoutant plus de contrainte.

## Énoncé du problème

La STS souhaite toujours ajouter cinq autobus sur cinq lignes différentes afin de répondre à la demande de sa clientèle. Ces autobus seront ajoutés aux départs de 08:00 et 08:30. Toutefois, certains trajets se chevauchent, et il vous est demandé de diviser les autobus en deux groupes pour minimiser ces chevauchements. L’objectif est que deux bus partageant des arrêts aient, dans la mesure du possible, des heures de départ différentes.

### Poids des chevauchements

Tous les chevauchements entre les lignes ne sont pas égaux. Un poids vous est donné pour chaque chevauchement, exprimé en nombre d'arrêts communs entre deux lignes. Voici le tableau des poids :

|             | **Ligne 7** | **Ligne 12** | **Ligne 16** | **Ligne 11** | **Ligne 3** |
|-------------|-------------|--------------|-------------|-------------|------------|
| **Ligne 7** |      -      |      10      |      0      |      11     |     13     |
| **Ligne 12**|      10     |       -      |      11     |      0      |     10     |
| **Ligne 16**|      0      |      11      |      -      |      8      |      0     |
| **Ligne 11**|      11     |       0      |      8      |      -      |      0     |
| **Ligne 3** |      13     |      10      |      0      |      0      |      -     |

Jusque là, cela demeurre similaire aux problèmes 1 et 2. 

### Nouvelle contrainte

Une nouvelle information vous est fournie : un même employé s'occupera **exceptionnellement** des lignes **3** et **12**. Par conséquent, ces deux lignes doivent impérativement être placées dans des groupes séparés, c'est-à-dire que leurs départs ne peuvent avoir lieu en même temps. Cela ajoute une **contrainte** à prendre en compte dans votre solution.

## Ajout de contraintes à l'hamiltonien - Un exemple
Posons, à des fins de démonstration, un problème à $N=2$ qubits, et un hamiltonien simple

$$
    H = \sum_i^N Z_i.
$$

Ce dernier peut être écrit en `SparsePauliOp`.

In [2]:
from qiskit.quantum_info import SparsePauliOp

H = SparsePauliOp.from_list([("IZ", 1.0), ("ZI", 1.0)])

Considérons maintenant la contrainte d'égalité suivante :

$$
Z_0 = 1
$$

Cela signifie que le qubit 0 doit nécessairement être dans l'état $\left|0\right>$, puisque la valeur moyenne de l'observable $Z_0$ ne sera égale à $1$ que lorsque le qubit est dans cet état.

### Transformation de la contrainte d'égalité

Pour intégrer cette contrainte dans l'hamiltonien, nous la réécrivons sous forme d'une expression de distance.

On commence par reformuler l'expression plus haut
$$
Z_0 - 1 = 0, 
$$

Trouvons maintenant notre expression de distance
$$
(Z_0 - 1)^2
$$

Écrivons la avec un `SparsePauliOp`.  

In [3]:
contrainte = (SparsePauliOp.from_list([("IZ", 1.0), ("II", -1.0)])) ** 2

Cette expression mesure la distance entre le résultat de la mesure de l'observable $Z_0$ et la valeur $1$. Si cette distance est égale à zéro, cela signifie que la contrainte $Z_0 = 1$ est satisfaite. La mise au carré de cette expression permet de garder une distance toujours positive, ce qui sera utile lors de l'ajout de cette contrainte à notre fonction de coût (hamiltonien). 

### Tolérance et pénalité associée à la contrainte

Pour indiquer notre tolérance au non-respect de cette contrainte, nous introduisons un paramètre $\alpha$, qui représente la pénalité associée au bris de la contrainte. Ce paramètre $\alpha$ devient un hyperparamètre de votre optimisation et viens multiplier l'expression de distance. Par exemple, pour un hamiltonien $H$, l'expression contrainte de ce dernier est obtenu par 

$$
    H_c = H + \alpha(Z_0 - 1)^2.
$$

In [4]:
alpha = 1
H_c = H + alpha * contrainte



Nous voulons un $\alpha$ tel que le fait de briser la contrainte $Z_0 = 1$ soit assez couteux pour que notre processus d'optimisation choisisse de ne pas le faire. 

Il est recommandé de commencer avec une valeur d’$\alpha$ légèrement supérieure à la somme des poids dans la fonction de coût sans contrainte. Cependant, un $\alpha$ trop élevé pourrait rendre l'algorithme QAOA instable, donc une attention particulière doit être portée à cet équilibre.

## Objectif

Votre objectif est de minimiser la somme des poids de chevauchement au sein de chaque groupe tout en respectant la **contrainte** de séparation des lignes 3 et 12. Le but est donc de séparer les départs des lignes avec plusieurs arrêts communs, tout en séparant systématiquement deux des autobus. 

### Livrables attendus :
- Un hamiltonien `SparsePauliOp` décrivant le problème contraint. 
- Une série de paramètres optimaux pour le circuit QAOA.
- Le nombre de couches du circuit QAOA.

#### Format de soumission des livrables :
Un fichier de format `.npz` est attendu. Vous pouvez générer ce fichier avec les informations requises en utilisation la méthode `sauvegarder_res` disponible dans `utils.py`.

Donnez un **nom significatif** à votre fichier, par exemple `equipe_A_probleme_3.npz`. Vous pouvez vérifier le contenu de votre fichier `.npz` à l'aide de la méthode `lire_res` disponible dans `utils.py`.

### Évaluation
Une partie de la correction se fera en exécutant votre circuit QAOA avec le nombre de couches et les paramètres fournis. Un score sera calculé à partir de la solution obtenue à l'aide de la méthode de notation prédéfinie `calc_score` disponible dans `utils.py`.

Les juges évalueront également la qualité du code et l'originalité de votre optimisation de paramètres pour QAOA. 

In [5]:
# Instanciation de votre Hamiltonien

In [6]:
# Instanciation de votre circuit de QAOA

In [7]:
# Optimisation de vos hyperparamètres

In [8]:
# Analyse et sauvegarde de vos résultats